In [2]:
import os
from dotenv import load_dotenv
import sys

# streamlit run src/frontend.py
# ##################################### IN ORDER TO IMPORT THE PACKAGGE FOR CONDA ENVIORNMENT
# Get the current directory of the script
script_dir = os.getcwd()

# Navigate up one level to the parent directory
project_dir = os.path.abspath(os.path.join(script_dir, '..'))

# Add the parent directory to sys.path
sys.path.append(project_dir)
# #####################################  FOR CONDA ENVIORNMENT 

# script_dir = os.getcwd()
# sys.path.append(os.path.abspath(os.path.join(script_dir, '..')))

from src.paths import PARENT_DIR

In [3]:
import hopsworks

In [4]:
print(hopsworks.__version__)

3.4.2


In [5]:
# load key-value pairs from .env file located in the parent directory
load_dotenv(PARENT_DIR / '.env')

HOPSWORKS_API_KEY = os.environ['HOPSWORKS_API_KEY']
HOPSWORKS_PROJECT_NAME = 'taxi_demand_calvin'

In [6]:

from datetime import datetime
import pandas as pd
from src.data import load_raw_data

from_year = 2022
to_year = 2023 #datetime.now().year
print(f'Downloading raw data from {from_year} to {to_year}')

rides = pd.DataFrame()
for year in range(from_year, to_year+1):
    
    # download data for the whole year
    rides_one_year = load_raw_data(year)
    
    # append rows
    rides = pd.concat([rides, rides_one_year])

File 2022-01 was already in local storage


File 2022-02 was already in local storage


File 2022-03 was already in local storage


File 2022-04 was already in local storage


File 2022-05 was already in local storage


...

File 2023-08 was already in local storage


File 2023-09 was already in local storage


File 2023-10 was already in local storage


File 2023-11 was already in local storage


File 2023-12 was already in local storage


In [23]:
print(f'{len(rides)=:,}')

len(rides)=0


In [7]:
from src.data import transform_raw_data_into_ts_data

ts_data = transform_raw_data_into_ts_data(rides)

100%|██████████| 265/265 [00:08<00:00, 31.30it/s]


In [17]:
# rides

In [49]:
# IcjMMvdfzZBOlCbE.mGsaeXANM7Y0zDxEwNUIRYkBqoEFLntZnC2oxTCRfbAfM1KGrf9C919b3XrFYECI

In [66]:
# HOPSWORKS_API_KEY='IcjMMvdfzZBOlCbE.mGsaeXANM7Y0zDxEwNUIRYkBqoEFLntZnC2oxTCRfbAfM1KGrf9C919b3XrFYECI'

In [8]:
project = hopsworks.login(
    project=HOPSWORKS_PROJECT_NAME,
    api_key_value=HOPSWORKS_API_KEY
)

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/578708


In [9]:
feature_store = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.


In [10]:
FEATURE_GROUP_NAME = 'time_series_hourly_feature_group'
FEATURE_GROUP_VERSION = 3 # for xgboost model training no

In [11]:
feature_group = feature_store.get_or_create_feature_group(
    name=FEATURE_GROUP_NAME,
    version=FEATURE_GROUP_VERSION,
    description="Time-series data at hourly frequency",
    primary_key = ['pickup_location_id', 'pickup_hour'],
    event_time='pickup_hour',
)

In [14]:
ts_data

,pickup_hour,rides,pickup_location_id
0,2022-01-01 00:00:00,0,1
1,2022-01-01 01:00:00,0,1
2,2022-01-01 02:00:00,0,1
3,2022-01-01 03:00:00,0,1
4,2022-01-01 04:00:00,1,1
...,...,...,...
4642795,2023-12-31 19:00:00,0,265
4642796,2023-12-31 20:00:00,0,265
4642797,2023-12-31 21:00:00,3,265
4642798,2023-12-31 22:00:00,3,265


In [12]:
feature_group.insert(ts_data, write_options={"wait_for_job": False})

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/578708/fs/574531/fg/683348


Uploading Dataframe: 0.00% |          | Rows 0/4642800 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: time_series_hourly_feature_group_3_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/578708/jobs/named/time_series_hourly_feature_group_3_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x18b5a5460>, None)

In [1]:
import hopsworks